In [1]:
import json
from pathlib import Path
import pandas as pd

filepath = Path('peeringdb/peeringdb_2_dump_2025_10_21.json')

with filepath.open('r', encoding='utf-8') as f:
    dump = json.load(f)

# extract the net.data section and load into a DataFrame
net_data = dump.get('net', {}).get('data')
if net_data is None:
    raise KeyError("JSON does not contain 'net' -> 'data' structure")

net_df = pd.DataFrame(net_data)
net_df['asn'] = net_df['asn'].astype(int)
net_df = net_df[net_df['info_type'] != '']

# show a quick preview
net_df.head()

,id,org_id,name,aka,name_long,website,social_media,asn,looking_glass,route_server,...,policy_ratio,policy_contracts,allow_ixp_update,status_dashboard,rir_status,rir_status_updated,logo,created,updated,status
0,1,8897,GTT Communications (AS4436),Formerly known as nLayer Communications,,http://www.gtt.net,"[{'service': 'website', 'identifier': 'http://...",4436,,,...,True,Required,False,None,ok,2024-06-26T04:47:55Z,None,2004-07-28T00:00:00Z,2022-07-27T05:33:22Z,ok
1,2,14,Akamai Technologies,,,https://www.akamai.com/,"[{'service': 'website', 'identifier': 'https:/...",20940,,,...,False,Not Required,False,https://www.akamaistatus.com/,ok,2024-06-26T04:47:55Z,None,2004-07-28T00:00:00Z,2025-10-20T12:16:12Z,ok
2,3,17,DALnet IRC Network,,,http://www.dal.net,"[{'service': 'website', 'identifier': 'http://...",31800,,,...,False,Not Required,False,,ok,2024-06-26T04:47:55Z,None,2004-07-28T00:00:00Z,2025-01-09T13:42:07Z,ok
3,5,9350,Swisscom,IP-Plus,,http://www.swisscom.com,"[{'service': 'website', 'identifier': 'http://...",3303,,telnet://route-server.ip-plus.net,...,True,Required,False,,ok,2024-06-26T04:47:55Z,None,2004-07-28T00:00:00Z,2025-08-12T06:33:30Z,ok
4,6,23,Cox Communications,Cox Communications,,http://www.cox.com/peering,"[{'service': 'website', 'identifier': 'http://...",22773,,,...,False,Required,False,,ok,2024-06-26T04:47:55Z,None,2004-07-28T00:00:00Z,2022-11-28T22:55:17Z,ok


In [2]:
import io
with open('/workspaces/pytorch-gpu-2/preprocessing/data/caida/20251001.as-org2info.txt', 'r', newline='', encoding='utf-8') as input_file:
    lines = input_file.readlines()   
    # Buffers initialisieren
    aut_lines = []
    org_lines = []
    mode = None
    total_lines = len(lines)
    aut_count = 0
    org_count = 0 

    for i, line in enumerate(lines):
        line = line.strip()
        if line.startswith("# format:aut"):
            mode = "aut"
            continue
        elif line.startswith("# format:org_id"):
            mode = "org"
            continue
        elif line.startswith("#") or not line:
            # Andere Kommentar- oder Leerzeilen überspringen
            continue      
        if mode == "aut":
            aut_lines.append(line)
            aut_count += 1
        elif mode == "org":
            org_lines.append(line)
            org_count += 1
    # StringIO-Objekte aus den gesammelten Zeilen bauen
    aut_buffer = io.StringIO("\n".join(aut_lines))
    org_buffer = io.StringIO("\n".join(org_lines))
    # DataFrames einlesen
    aut_df = pd.read_csv(aut_buffer, sep="|",
                        names=["aut", "changed", "aut_name", "org_id", "opaque_id", "source"], usecols=["aut", "org_id", "source", "changed"])
    org_df = pd.read_csv(org_buffer, sep="|",
                        names=["org_id", "changed", "org_name", "country", "source"], usecols=["org_id", "org_name", "country"])

    # Join the DataFrames
    joined_df = pd.merge(aut_df, org_df, on="org_id", how="left")
joined_df.head()

,aut,changed,org_id,source,org_name,country
0,1,20240618.0,LPL-141-ARIN,ARIN,"Level 3 Parent, LLC",US
1,2,20231108.0,UNIVER-19-Z-ARIN,ARIN,University of Delaware,US
2,3,20100927.0,MIT-2-ARIN,ARIN,Massachusetts Institute of Technology,US
3,4,20230929.0,USC-32-Z-ARIN,ARIN,University of Southern California,US
4,5,20200723.0,WGL-117-ARIN,ARIN,WFA Group LLC,US


In [3]:
peering_df_joined = pd.merge(net_df, joined_df, left_on='asn', right_on='aut', how='left')
peering_df_joined = peering_df_joined[['asn', 'org_name', 'country', 'source', 'info_type']]
peering_df_joined.head()

,asn,org_name,country,source,info_type
0,4436,"GTT Americas, LLC",US,ARIN,NSP
1,20940,Akamai International B.V.,NL,RIPE,Content
2,31800,DALnet,US,ARIN,Non-Profit
3,3303,Swisscom (Schweiz) AG,CH,RIPE,Cable/DSL/ISP
4,22773,Cox Communications Inc.,US,ARIN,Cable/DSL/ISP


In [4]:
import requests
import json
import pandas as pd
next_page = True
nodes = []
first=5000
offset=0
while(next_page):
    print(f"Fetching AS Rank data: first={first}, offset={offset}")
    response = requests.get(f"https://api.asrank.caida.org/v2/restful/asns/?first={first}&offset={offset}")

    if response.status_code != 200:
        raise ValueError(f"Request failed with status code {response.status_code}")

    as_rank_dump = response.json()
    nodes.extend(as_rank_dump['data']['asns']['edges'])
    if not as_rank_dump['data']['asns']['pageInfo']['hasNextPage']:
        next_page = False
    offset += first

edges = [e['node'] for e in nodes]
as_rank_df = pd.DataFrame(edges)
# 1️⃣ asnDegree (dict) in eigene Spalten auflösen
asnDegree_df = as_rank_df["asnDegree"].apply(pd.Series)
asnDegree_df.columns = [f"asnDegree_{c}" for c in asnDegree_df.columns]

# 2️⃣ wieder an den Haupt-DataFrame anhängen
as_rank_df = pd.concat([as_rank_df.drop(columns=["asnDegree"]), asnDegree_df], axis=1)

# 1️⃣ asnDegree (dict) in eigene Spalten auflösen
asnCone_df = as_rank_df["cone"].apply(pd.Series)
asnCone_df.columns = [f"cone_{c}" for c in asnCone_df.columns]

# 2️⃣ wieder an den Haupt-DataFrame anhängen
as_rank_df = pd.concat([as_rank_df.drop(columns=["cone"]), asnCone_df], axis=1)

# show a quick preview
as_rank_df

Fetching AS Rank data: first=5000, offset=0


KeyboardInterrupt: 

In [6]:
as_rank_df = pd.read_csv('/workspaces/pytorch-gpu-2/preprocessing/data/as_rank_df.csv')
as_rank_df.head()

,asn,rank,asnDegree_total,asnDegree_customer,asnDegree_peer,asnDegree_provider,cone_numberAsns,cone_numberPrefixes,cone_numberAddresses
0,3356,1,6613,6545,68,0,53986,873410,3468642119
1,1299,2,2567,2509,58,0,41193,776707,3219679484
2,174,3,6723,6626,97,0,38887,730166,3034352967
3,3257,4,1853,1816,37,0,36040,612491,2791999209
4,2914,5,1541,1483,58,0,25179,576134,2918763154


In [11]:
peering_df_joined_with_asrank = pd.merge(
    peering_df_joined,
    as_rank_df,
    left_on='asn',
    right_on='asn',
    how='left'
)
peering_df_joined_with_asrank = peering_df_joined_with_asrank.dropna()
peering_df_joined_with_asrank

,asn,org_name,country,source,info_type,rank,asnDegree_total,asnDegree_customer,asnDegree_peer,asnDegree_provider,cone_numberAsns,cone_numberPrefixes,cone_numberAddresses
0,4436,"GTT Americas, LLC",US,ARIN,NSP,78320.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,20940,Akamai International B.V.,NL,RIPE,Content,1894.0,485.0,14.0,366.0,105.0,15.0,8945.0,14612752.0
2,31800,DALnet,US,ARIN,Non-Profit,47745.0,78.0,0.0,74.0,4.0,1.0,2.0,512.0
3,3303,Swisscom (Schweiz) AG,CH,RIPE,Cable/DSL/ISP,81.0,1273.0,166.0,1101.0,6.0,733.0,22131.0,42899794.0
4,22773,Cox Communications Inc.,US,ARIN,Cable/DSL/ISP,110.0,499.0,489.0,8.0,2.0,505.0,11982.0,31992440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23620,35359,ADR TEL s.p.a.,IT,RIPE,Cable/DSL/ISP,26962.0,2.0,0.0,0.0,2.0,1.0,8.0,2048.0
23621,26431,Happy Telecommunications LLC,US,ARIN,Content,78320.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
23623,210856,Jesse Bakker trading as Skynode,NL,RIPE,NSP,66977.0,1.0,0.0,0.0,1.0,1.0,1.0,256.0
23624,152033,PT Mandari Teknologi Nusantara,ID,APNIC,Cable/DSL/ISP,78320.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [14]:
# ==== Text + numerische AS-Rank-Features: robustes, korrigiertes Training ====
import os, numpy as np, pandas as pd, torch, torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import (
    AutoTokenizer, AutoModel, TrainingArguments, Trainer, EarlyStoppingCallback, AdamW
)
import joblib

# --------- Konfig ---------
MODEL_NAME   = "xlm-roberta-base"
MAX_LENGTH   = 64
LR_BACKBONE  = 5e-6          # Basis-LR für den Backbone nach Unfreeze
LR_HEAD      = 2e-4          # höhere LR für Kopf/Num-Proj beim Warmup; nach Unfreeze kleiner
EPOCHS_WARM  = 2             # 1–2 Epochen: nur Kopf trainieren
EPOCHS_MAIN  = 8             # 6–10: gesamtes Modell mit LLRD
BATCH_SIZE   = 32
WARMUP_RATIO = 0.06
SEED         = 100
OUT_DIR      = "xlmr_org_trainer_out_mixed_fix"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

# --------- Daten ---------
df = peering_df_joined_with_asrank.copy()
df = df.reset_index(drop=True)   # Index zurücksetzen

# Label
le = LabelEncoder()
df["label_id"] = le.fit_transform(df["info_type"].astype(str))
num_labels = len(le.classes_)
id2label = {i: c for i,c in enumerate(le.classes_)}
label2id = {c: i for i,c in enumerate(le.classes_)}

# Text
def norm_name(s: str) -> str:
    s = str(s).strip().lower()
    s = " ".join(s.split())
    return s or "unknown"
df["org_name"] = df["org_name"].fillna("unknown").astype(str).map(norm_name)

# Numerische Features
FEAT_COLS = [
    "rank",
    "asnDegree_total","asnDegree_customer","asnDegree_peer","asnDegree_provider",
    "cone_numberAsns","cone_numberPrefixes","cone_numberAddresses",
]
num_df = df[FEAT_COLS].copy()
for c in FEAT_COLS:
    v = pd.to_numeric(num_df[c], errors="coerce").fillna(0)
    # starke Schiefe log1p-transformieren
    if c.startswith("asnDegree") or c.startswith("cone_number") or c == "rank":
        num_df[c] = np.log1p(v)
    else:
        num_df[c] = v

# Split (stratifiziert)
train_idx, eval_idx = train_test_split(
    np.arange(len(df)),
    test_size=0.13,
    random_state=SEED,
    stratify=df["label_id"]
)
y_train_np  = df.loc[train_idx, "label_id"].to_numpy()
y_eval_np   = df.loc[eval_idx,  "label_id"].to_numpy()

texts_train = df.loc[train_idx, "org_name"].tolist()
texts_eval  = df.loc[eval_idx,  "org_name"].tolist()

Xnum_train_raw = num_df.loc[train_idx].to_numpy(dtype=np.float32)
Xnum_eval_raw  = num_df.loc[eval_idx].to_numpy(dtype=np.float32)

# Standardisieren: nur auf Train fitten!
scaler = StandardScaler()
Xnum_train = scaler.fit_transform(Xnum_train_raw)
Xnum_eval  = scaler.transform(Xnum_eval_raw)

# Tokenizer & Encodings (Padding übernimmt Collator)
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
enc_train = tok(texts_train, truncation=True, max_length=MAX_LENGTH)
enc_eval  = tok(texts_eval,  truncation=True, max_length=MAX_LENGTH)

# Dataset
class TextNumDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, num_feats, labels):
        self.enc = encodings
        self.num = num_feats
        self.y   = labels
    def __len__(self): return len(self.y)
    def __getitem__(self, idx):
        item = {k: self.enc[k][idx] for k in self.enc}  # plain Python lists
        item["features"] = self.num[idx]                # np.ndarray row
        item["labels"]   = int(self.y[idx])
        return item

ds_train = TextNumDataset(enc_train, Xnum_train, y_train_np)
ds_eval  = TextNumDataset(enc_eval,  Xnum_eval,  y_eval_np)

# Collator
class MixedCollator:
    def __init__(self, tokenizer): self.tok = tokenizer
    def __call__(self, batch):
        text = {k: [b[k] for b in batch] for k in ["input_ids","attention_mask"]}
        text = self.tok.pad(text, return_tensors="pt")
        text["features"] = torch.tensor([b["features"] for b in batch], dtype=torch.float)
        text["labels"]   = torch.tensor([b["labels"] for b in batch], dtype=torch.long)
        return text

collator = MixedCollator(tok)

# Modell
class TextPlusNumClassifier(nn.Module):
    def __init__(self, model_name, num_labels, num_num_feats, dropout=0.2):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(model_name)
        hidden = self.backbone.config.hidden_size  # 768 für XLM-R base

        # Numerik-Projektion (gibt den Zahlen „Stimme“)
        self.num_proj = nn.Sequential(
            nn.Linear(num_num_feats, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.LayerNorm(128),
            nn.Dropout(0.1),
        )

        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Sequential(
            nn.Linear(hidden + 128, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, num_labels),
        )

    def forward(self, input_ids=None, attention_mask=None, features=None, labels=None):
        out = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        # CLS-Pooling (XLM-R: <s> am Index 0)
        pooled = out.last_hidden_state[:, 0]                  # [B,H]
        num_emb = self.num_proj(features)                     # [B,128]
        z = torch.cat([pooled, num_emb], dim=1)               # [B,H+128]
        logits = self.classifier(self.dropout(z))
        loss = None
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)      # erst mal „pur“
        return {"loss": loss, "logits": logits}

model = TextPlusNumClassifier(MODEL_NAME, num_labels=num_labels, num_num_feats=Xnum_train.shape[1]).to(device)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy":  float(accuracy_score(labels, preds)),
        "f1_macro":  float(f1_score(labels, preds, average="macro")),
        "precision": float(precision_score(labels, preds, average="macro", zero_division=0)),
        "recall":    float(recall_score(labels, preds, average="macro")),
    }

# ---------------------------
# 1) HEAD-ONLY WARMUP (Backbone gefroren)
# ---------------------------
for p in model.backbone.parameters():  # Backbone einfrieren
    p.requires_grad = False

args_warm = TrainingArguments(
    output_dir=OUT_DIR + "/warmup",
    num_train_epochs=EPOCHS_WARM,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR_HEAD,                 # Kopf darf schneller lernen
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    warmup_ratio=WARMUP_RATIO,
    fp16=False,                            # stabil
    weight_decay=0.01,
    logging_steps=100,
    seed=SEED,
    report_to=["none"],
)

trainer_warm = Trainer(
    model=model,
    args=args_warm,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    tokenizer=tok,
    data_collator=collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

print("\n=== Head-only Warmup ===")
trainer_warm.train()
print("Warmup Eval:", trainer_warm.evaluate())

# ---------------------------
# 2) UNFREEZE + LLRD (Layer-wise LR Decay)
# ---------------------------
for p in model.backbone.parameters():
    p.requires_grad = True

# LLRD: untere Layer kleinere LR, Head größere LR
param_groups = []
nL = model.backbone.config.num_hidden_layers
# Encoder-Layer
for i in range(nL):
    params_i = [p for n,p in model.backbone.named_parameters() if f"encoder.layer.{i}." in n]
    if params_i:
        param_groups.append({"params": params_i, "lr": LR_BACKBONE * (0.9 ** (nL - 1 - i))})
# Embeddings
emb_params = [p for n,p in model.backbone.named_parameters() if "embeddings" in n]
if emb_params:
    param_groups.append({"params": emb_params, "lr": LR_BACKBONE * (0.9 ** nL)})
# Kopf (num_proj + classifier) mit höherer LR
head_params = [p for n,p in model.named_parameters() if ("num_proj" in n) or ("classifier" in n)]
param_groups.append({"params": head_params, "lr": LR_BACKBONE * 8})

optimizer = AdamW(param_groups, weight_decay=0.01)

args_main = TrainingArguments(
    output_dir=OUT_DIR + "/checkpoints",
    num_train_epochs=EPOCHS_MAIN,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LR_BACKBONE,            # wird von param_groups überschrieben
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    warmup_ratio=WARMUP_RATIO,
    fp16=False,
    weight_decay=0.01,
    logging_steps=100,
    seed=SEED,
    report_to=["none"],
)

trainer = Trainer(
    model=model,
    args=args_main,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    tokenizer=tok,
    data_collator=collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    optimizers=(optimizer, None),          # <- unser LLRD-Optimizer
)

print("\n=== Unfreeze + LLRD ===")
trainer.train()
print("Final Eval:", trainer.evaluate())

# Speichern
os.makedirs(OUT_DIR + "/model", exist_ok=True)
trainer.save_model(OUT_DIR + "/model")
tok.save_pretrained(OUT_DIR + "/model")
joblib.dump({"scaler": scaler, "feat_cols": FEAT_COLS, "label_encoder": le}, OUT_DIR + "/model/aux.pkl")
print("Gespeichert nach:", OUT_DIR + "/model")


Device: cuda


/home/vscode/.local/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



=== Head-only Warmup ===


/tmp/ipykernel_2328/2740863333.py:107: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:261.)
  text["features"] = torch.tensor([b["features"] for b in batch], dtype=torch.float)


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision,Recall
1,1.498000,1.421169,0.512989,0.099231,0.145458,0.117807
2,1.430300,1.402779,0.518251,0.103131,0.154370,0.119113


Warmup Eval: {'eval_loss': 1.4027788639068604, 'eval_accuracy': 0.5182505754685959, 'eval_f1_macro': 0.10313052987034874, 'eval_precision': 0.15437042039320942, 'eval_recall': 0.11911256766115803, 'eval_runtime': 1.0431, 'eval_samples_per_second': 2915.32, 'eval_steps_per_second': 92.032, 'epoch': 2.0}

=== Unfreeze + LLRD ===


/home/vscode/.local/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/vscode/.local/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision,Recall
1,1.341600,1.278851,0.559355,0.211321,0.270957,0.216799
2,1.292900,1.224119,0.573167,0.276170,0.345887,0.270960
3,1.229400,1.196620,0.587636,0.296089,0.328967,0.298595
4,1.210500,1.184992,0.583690,0.345557,0.394055,0.335610
5,1.170000,1.183088,0.588622,0.366560,0.394769,0.367987
6,1.196200,1.172963,0.595199,0.374609,0.408676,0.368994
7,1.175400,1.173576,0.597501,0.377181,0.404846,0.378122
8,1.153700,1.173249,0.596514,0.376048,0.403539,0.376960


Final Eval: {'eval_loss': 1.1735758781433105, 'eval_accuracy': 0.597500822097994, 'eval_f1_macro': 0.3771814000111374, 'eval_precision': 0.4048462755629164, 'eval_recall': 0.37812229245071194, 'eval_runtime': 1.1219, 'eval_samples_per_second': 2710.694, 'eval_steps_per_second': 85.573, 'epoch': 8.0}
Gespeichert nach: xlmr_org_trainer_out_mixed_fix/model


In [20]:
# ==== Light Ensembling: HF (Text+Num) + XGBoost/RandomForest auf Numeric ====
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 1) HF-Probabilitäten (Softmax über Logits)
pred = trainer.predict(ds_eval)  # enthält .predictions (logits) & .label_ids
logits = pred.predictions
probs_hf = np.exp(logits - logits.max(axis=1, keepdims=True))
probs_hf = probs_hf / probs_hf.sum(axis=1, keepdims=True)
y_true   = pred.label_ids

# 2) Zweitmodell auf numerischen Features: XGBoost, sonst RandomForest
clf_name = None

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(
    n_estimators=800, max_depth=None,
    class_weight="balanced_subsample", n_jobs=-1, random_state=42
)
clf_name = "RandomForest"

clf.fit(Xnum_train, y_train_np)

# 2b) Ensure: proba-Aligment auf die globale Klassenreihenfolge
def probs_aligned(clf, X, global_classes):
    """Align predict_proba Spalten auf global_classes-Reihenfolge."""
    p = clf.predict_proba(X)
    # sklearn/xgb liefern eigene class_ Ordnung:
    model_classes = clf.classes_
    out = np.zeros((p.shape[0], len(global_classes)), dtype=float)
    colmap = {c:i for i,c in enumerate(model_classes)}
    for j, c in enumerate(global_classes):
        if c in colmap:
            out[:, j] = p[:, colmap[c]]
        else:
            # falls Klasse im Train fehlte: 0-Proba
            out[:, j] = 0.0
    # numerische Stabilität
    row_sum = out.sum(axis=1, keepdims=True)
    mask = row_sum.squeeze(-1) > 0
    out[mask] /= row_sum[mask]
    # falls Zeile Summe=0 (extremer Randfall): gleich verteilen
    out[~mask] = 1.0/len(global_classes)
    return out

probs_num = probs_aligned(clf, Xnum_eval, global_classes=np.arange(len(le.classes_)))

# 3) Baselines drucken
acc_hf  = accuracy_score(y_true, probs_hf.argmax(1))
f1m_hf  = f1_score(y_true, probs_hf.argmax(1), average="macro")
acc_num = accuracy_score(y_true, probs_num.argmax(1))
f1m_num = f1_score(y_true, probs_num.argmax(1), average="macro")

print(f"HF (Text+Num-Netz)     : Acc={acc_hf:.4f}  F1m={f1m_hf:.4f}")
print(f"{clf_name:>18s} only : Acc={acc_num:.4f}  F1m={f1m_num:.4f}")

# 4) Gewichtetes Mischen (alpha = Anteil HF)
alphas = [0.9, 0.85, 0.8, 0.75, 0.7, 0.6, 0.5]
best = (-1, -1, None)  # (acc, f1m, alpha)
for a in alphas:
    probs_ens = a*probs_hf + (1-a)*probs_num
    y_pred = probs_ens.argmax(1)
    acc = accuracy_score(y_true, y_pred)
    f1m = f1_score(y_true, y_pred, average="macro")
    print(f"Ensemble a={a:.2f}     : Acc={acc:.4f}  F1m={f1m:.4f}")
    if (acc, f1m) > (best[0], best[1]):
        best = (acc, f1m, a)

print("\nBest Ensemble:")
print(f"a={best[2]:.2f}  Acc={best[0]:.4f}  F1m={best[1]:.4f}")

# Optional: detaillierter Report des besten Mischgewichts
a = best[2]
probs_best = a*probs_hf + (1-a)*probs_num
y_pred_best = probs_best.argmax(1)
print("\nPer-Class Report (best a):")
print(classification_report(y_true, y_pred_best, target_names=list(le.classes_), zero_division=0))


HF (Text+Num-Netz)     : Acc=0.5975  F1m=0.3772
      RandomForest only : Acc=0.3907  F1m=0.1654
Ensemble a=0.90     : Acc=0.6014  F1m=0.3773
Ensemble a=0.85     : Acc=0.6014  F1m=0.3763
Ensemble a=0.80     : Acc=0.6041  F1m=0.3764
Ensemble a=0.75     : Acc=0.6057  F1m=0.3751
Ensemble a=0.70     : Acc=0.6057  F1m=0.3767
Ensemble a=0.60     : Acc=0.5978  F1m=0.3667
Ensemble a=0.50     : Acc=0.5939  F1m=0.3411

Best Ensemble:
a=0.70  Acc=0.6057  F1m=0.3767

Per-Class Report (best a):
                      precision    recall  f1-score   support

       Cable/DSL/ISP       0.65      0.90      0.76      1521
             Content       0.41      0.34      0.37       320
Educational/Research       0.63      0.74      0.68       184
          Enterprise       0.45      0.24      0.32       221
          Government       0.56      0.62      0.59        16
                 NSP       0.55      0.20      0.30       515
    Network Services       0.00      0.00      0.00       102
          Non-Pr

In [21]:
# ==== Stacking: Meta-LogReg lernt Kombination aus HF- und Numeric-Modell ====
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
import joblib

# 0) Helper: Spalten der predict_proba an globale Klassenordnung angleichen
def probs_aligned(clf, X, global_classes):
    p = clf.predict_proba(X)
    model_classes = clf.classes_
    out = np.zeros((p.shape[0], len(global_classes)), dtype=float)
    colmap = {c:i for i,c in enumerate(model_classes)}
    for j, c in enumerate(global_classes):
        if c in colmap:
            out[:, j] = p[:, colmap[c]]
        else:
            out[:, j] = 0.0
    row_sum = out.sum(axis=1, keepdims=True)
    mask = row_sum.squeeze(-1) > 0
    out[mask] /= row_sum[mask]
    out[~mask] = 1.0/len(global_classes)
    return out

# 1) HF-Wahrscheinlichkeiten (Train + Eval)
pred_tr = trainer.predict(ds_train)
pred_ev = trainer.predict(ds_eval)

logits_tr = pred_tr.predictions
logits_ev = pred_ev.predictions

# numerisch stabile Softmax
probs_hf_tr = np.exp(logits_tr - logits_tr.max(axis=1, keepdims=True))
probs_hf_tr /= probs_hf_tr.sum(axis=1, keepdims=True)

probs_hf_ev = np.exp(logits_ev - logits_ev.max(axis=1, keepdims=True))
probs_hf_ev /= probs_hf_ev.sum(axis=1, keepdims=True)

y_true_tr = pred_tr.label_ids
y_true_ev = pred_ev.label_ids

# 2) Zweitmodell auf Numeric-Features (nimm dein vorhandenes; sonst RF)
try:
    clf  # falls du schon einen (XGB/RF) aus dem vorherigen Schritt hast
except NameError:
    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier(
        n_estimators=800,
        class_weight="balanced_subsample",
        n_jobs=-1,
        random_state=42
    )
    clf.fit(Xnum_train, y_train_np)

probs_num_tr = probs_aligned(clf, Xnum_train, global_classes=np.arange(len(le.classes_)))
probs_num_ev = probs_aligned(clf, Xnum_eval,  global_classes=np.arange(len(le.classes_)))

# 3) Meta-Features: concat der Basis-Wahrscheinlichkeiten
X_meta_tr = np.hstack([probs_hf_tr, probs_num_tr])
X_meta_ev = np.hstack([probs_hf_ev, probs_num_ev])

# 4) Meta-Learner (multinomial LogReg). Kein class_weight hier: Inputs sind schon Probabilitäten.
meta = LogisticRegression(
    max_iter=2000,
    multi_class="multinomial",
    solver="lbfgs"   # stabil für multinomial
)
meta.fit(X_meta_tr, y_true_tr)

y_pred_stack = meta.predict(X_meta_ev)

acc = accuracy_score(y_true_ev, y_pred_stack)
f1m = f1_score(y_true_ev, y_pred_stack, average="macro")
print(f"STACKING  Acc={acc:.4f}  F1m={f1m:.4f}")
print("\nPer-Class Report:")
print(classification_report(y_true_ev, y_pred_stack, target_names=list(le.classes_), zero_division=0))

# 5) Optional: speichern für Deployment
joblib.dump(
    {"meta": meta, "base_numeric": clf, "label_encoder": le},
    OUT_DIR + "/model/stacking.pkl"
)
print("Stacking artefacts saved to", OUT_DIR + "/model/stacking.pkl")


STACKING  Acc=0.5571  F1m=0.3238

Per-Class Report:
                      precision    recall  f1-score   support

       Cable/DSL/ISP       0.63      0.84      0.72      1521
             Content       0.32      0.28      0.30       320
Educational/Research       0.60      0.60      0.60       184
          Enterprise       0.36      0.22      0.27       221
          Government       0.57      0.25      0.35        16
                 NSP       0.39      0.23      0.29       515
    Network Services       0.06      0.01      0.02       102
          Non-Profit       0.31      0.13      0.18        78
     Route Collector       0.00      0.00      0.00         4
        Route Server       0.52      0.51      0.52        80

            accuracy                           0.56      3041
           macro avg       0.38      0.31      0.32      3041
        weighted avg       0.51      0.56      0.52      3041

Stacking artefacts saved to xlmr_org_trainer_out_mixed_fix/model/stacking.pkl

In [25]:
# ==== XGB auf [Text-Embeddings || Numeric] + Ensemble mit HF ====
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 1) Embeddings aus dem bereits trainierten Modell ziehen (CLS)
model.eval()
@torch.no_grad()
def get_embeds(dataset, batch=256):
    embs = []
    for i in range(0, len(dataset), batch):
        batch_items = [dataset[j] for j in range(i, min(i+batch, len(dataset)))]
        enc = tok.pad(
            {"input_ids":[b["input_ids"] for b in batch_items],
             "attention_mask":[b["attention_mask"] for b in batch_items]},
            return_tensors="pt"
        )
        enc = {k: v.to(device) for k, v in enc.items()}
        # Backbone forward
        out = model.backbone(**enc)
        cls = out.last_hidden_state[:, 0]          # CLS
        embs.append(cls.cpu().numpy())
    return np.vstack(embs)

Xtxt_tr = get_embeds(ds_train)
Xtxt_ev = get_embeds(ds_eval)

# 2) Kombinieren mit numerischen Features
Xcomb_tr = np.hstack([Xtxt_tr, Xnum_train])
Xcomb_ev = np.hstack([Xtxt_ev, Xnum_eval])

# 3) XGBoost (Fallback: RandomForest)
def probs_aligned(clf, X, global_classes):
    p = clf.predict_proba(X)
    model_classes = clf.classes_
    out = np.zeros((p.shape[0], len(global_classes)), dtype=float)
    colmap = {c:i for i,c in enumerate(model_classes)}
    for j, c in enumerate(global_classes):
        out[:, j] = p[:, colmap[c]] if c in colmap else 0.0
    s = out.sum(axis=1, keepdims=True)
    ok = s.squeeze(-1) > 0
    out[ok] /= s[ok]
    out[~ok] = 1.0 / len(global_classes)
    return out

clf_name = None
try:
    import xgboost as xgb
    clf = xgb.XGBClassifier(
        objective="multi:softprob",
        num_class=len(le.classes_),
        n_estimators=900, max_depth=8, learning_rate=0.06,
        subsample=0.9, colsample_bytree=0.9,
        reg_lambda=1.0, reg_alpha=0.0,
        tree_method="hist",            # falls GPU: "gpu_hist"
        n_jobs=-1, random_state=42
    )
    clf_name = "XGBoost[emb+num]"
except Exception:
    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier(
        n_estimators=800, class_weight="balanced_subsample",
        n_jobs=-1, random_state=42
    )
    clf_name = "RandomForest[emb+num]"

clf.fit(Xcomb_tr, y_train_np)
probs_xgb = probs_aligned(clf, Xcomb_ev, global_classes=np.arange(len(le.classes_)))

# 4) HF-Probabilitäten holen
pred_ev = trainer.predict(ds_eval)
logits_ev = pred_ev.predictions
probs_hf = np.exp(logits_ev - logits_ev.max(axis=1, keepdims=True))
probs_hf /= probs_hf.sum(axis=1, keepdims=True)
y_true = pred_ev.label_ids

# 5) Einzel-Performance
acc_x = accuracy_score(y_true, probs_xgb.argmax(1))
f1m_x = f1_score(y_true, probs_xgb.argmax(1), average="macro")
acc_h = accuracy_score(y_true, probs_hf.argmax(1))
f1m_h = f1_score(y_true, probs_hf.argmax(1), average="macro")
print(f"{clf_name:20s}: Acc={acc_x:.4f}  F1m={f1m_x:.4f}")
print(f"HF (Text+Num-Netz) : Acc={acc_h:.4f}  F1m={f1m_h:.4f}")

# 6) Ensemble (gewichtetes Mischen der Wahrscheinlichkeiten)
best = (-1, -1, None)
for a in [0.9,0.85,0.8,0.75,0.7,0.6]:
    probs_ens = a*probs_hf + (1-a)*probs_xgb
    y_pred = probs_ens.argmax(1)
    acc = accuracy_score(y_true, y_pred)
    f1m = f1_score(y_true, y_pred, average="macro")
    print(f"Ensemble a={a:.2f}    : Acc={acc:.4f}  F1m={f1m:.4f}")
    if (acc, f1m) > (best[0], best[1]):
        best = (acc, f1m, a)

print("\nBest Ensemble:")
print(f"a={best[2]:.2f}  Acc={best[0]:.4f}  F1m={best[1]:.4f}")

# 7) Report für das beste a
from sklearn.metrics import classification_report
probs_best = best[2]*probs_hf + (1-best[2])*probs_xgb
y_pred_best = probs_best.argmax(1)
print("\nPer-Class Report (best a):")
print(classification_report(y_true, y_pred_best, target_names=list(le.classes_), zero_division=0))


XGBoost[emb+num]    : Acc=0.6136  F1m=0.4356
HF (Text+Num-Netz) : Acc=0.5975  F1m=0.3772
Ensemble a=0.90    : Acc=0.6044  F1m=0.3878
Ensemble a=0.85    : Acc=0.6084  F1m=0.3961
Ensemble a=0.80    : Acc=0.6110  F1m=0.3981
Ensemble a=0.75    : Acc=0.6110  F1m=0.3979
Ensemble a=0.70    : Acc=0.6143  F1m=0.4034
Ensemble a=0.60    : Acc=0.6143  F1m=0.4024

Best Ensemble:
a=0.70  Acc=0.6143  F1m=0.4034

Per-Class Report (best a):
                      precision    recall  f1-score   support

       Cable/DSL/ISP       0.66      0.89      0.76      1521
             Content       0.44      0.36      0.40       320
Educational/Research       0.62      0.75      0.68       184
          Enterprise       0.42      0.24      0.31       221
          Government       0.50      0.69      0.58        16
                 NSP       0.50      0.24      0.33       515
    Network Services       0.50      0.03      0.06       102
          Non-Profit       0.59      0.24      0.35        78
     Route Co